In [7]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage, ToolMessage

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
search = DuckDuckGoSearchRun()

# Bind tool to LLM
llm_with_tools = llm.bind_tools([search])

def run_agent(query: str) -> str:
    messages = [HumanMessage(content=query)]
    
    # First LLM call
    response = llm_with_tools.invoke(messages)
    messages.append(response)
    
    # Check if tool call needed
    if response.tool_calls:
        for tool_call in response.tool_calls:
            result = search.invoke(tool_call["args"]["query"])
            messages.append(ToolMessage(content=result, tool_call_id=tool_call["id"]))
        
        # Final LLM call with tool results
        response = llm_with_tools.invoke(messages)
    
    return response.content

# Run
answer = run_agent("Who won the latest Nobel Prize in Economics?")
print(answer)

The latest Nobel Prize in Economics for 2023 was awarded to Claudia Goldin for her work in advancing the understanding of women's labor market outcomes. She is notable for being the third woman to receive this prestigious award.


In [8]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
search = DuckDuckGoSearchRun()

@tool
def calculate(expression: str) -> str:
    """Evaluate a math expression. Example: '2 + 2' or '15 * 7'."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

tools = [search, calculate]
tool_map = {t.name: t for t in tools}
llm_with_tools = llm.bind_tools(tools)

def run_agent(query: str, verbose: bool = True) -> str:
    messages = [HumanMessage(content=query)]
    
    response = llm_with_tools.invoke(messages)
    messages.append(response)
    
    while response.tool_calls:
        for tool_call in response.tool_calls:
            if verbose:
                print(f"🔧 Using: {tool_call['name']}({tool_call['args']})")
            result = tool_map[tool_call["name"]].invoke(tool_call["args"])
            if verbose:
                print(f"📄 Result: {str(result)[:200]}")
            messages.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"]))
        
        response = llm_with_tools.invoke(messages)
        messages.append(response)
    
    return response.content

# Test: requires both search and calculation
answer = run_agent("What is the population of Tokyo? Multiply that number by 2.")
print(f"\n✅ {answer}")

🔧 Using: duckduckgo_search({'query': 'current population of Tokyo 2023'})
📄 Result: 6 hours ago - Tokyo, officially the Tokyo Metropolis, is the capital and most populous city in Japan. With a population of over 14 million in the city proper in 2023, it is one of the most populous ur
🔧 Using: calculate({'expression': '14000000 * 2'})
📄 Result: 28000000

✅ The population of Tokyo is over 14 million. When multiplied by 2, the result is 28 million.
